In [10]:
import pandas as pd
import geopandas as gpd

from runner import calibration, scenario

import os
from os import listdir
from os.path import isfile, join
import pathlib

#Pandas display options to show more data when DataFrames are shown in the notebook (defaults are quite limited)
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300
pd.options.display.precision = 5

#pygeos and pyogrio can speed up geopandas but can also cause package dependency conflicts - disable if conflicts occur on your system.
#gpd.options.use_pygeos = True
import pyogrio #Only works if (engine = "pyogrio") is used in geopandas .to_file() code. 

In [11]:
scenarios = ["baseline", "improved_access", "ambitious_development"]
#scenarios = ["baseline"]

processed_input_files_directory = r"..\mled_processed_input_files"
processed_input_files_filenames = list(pathlib.Path(processed_input_files_directory).glob('*.csv'))
scenario_input_files_dictionary = dict(zip(scenarios, processed_input_files_filenames))

specification_files_directory = r"..\onsset_replication\onsset_input_files"
specification_files_filenames = list(pathlib.Path(specification_files_directory).glob('*.xlsx'))
scenario_specification_files_dictionary = dict(zip(scenarios, specification_files_filenames))

results_folder = r'..\results\onsset_full_results'
summary_folder = r'..\results\onsset_summary_results'
nest_summary_folder = r'..\results\nest_summaries'

In [12]:
scenario_input_files_dictionary


{'baseline': WindowsPath('../mled_processed_input_files/1_zambia_onsset_input_mled_demands_baseline.csv'),
 'improved_access': WindowsPath('../mled_processed_input_files/2_zambia_onsset_input_mled_demands_improved_access.csv'),
 'ambitious_development': WindowsPath('../mled_processed_input_files/3_zambia_onsset_input_mled_demands_ambitious_development.csv')}

In [13]:
scenario_specification_files_dictionary

{'baseline': WindowsPath('../onsset_replication/onsset_input_files/1_zambia_onsset_specifications_calibrated_baseline.xlsx'),
 'improved_access': WindowsPath('../onsset_replication/onsset_input_files/2_zambia_onsset_specifications_calibrated_improved_access.xlsx'),
 'ambitious_development': WindowsPath('../onsset_replication/onsset_input_files/3_zambia_onsset_specifications_calibrated_ambitious_development.xlsx')}

In [14]:
#WARNING: This should take quite some time (possibly 10 to 20 minutes depending on country size, computer speed, and number of scenarios run)

mled_scenario_number_counter = 1
for mled_scenario in scenarios:
    print("Running MLED scenario '" + mled_scenario +"'...")
    specs_path = os.path.join(scenario_specification_files_dictionary[mled_scenario])

    specs = pd.read_excel(specs_path, index_col=0)

    calibrated_csv_path = os.path.join(scenario_input_files_dictionary[mled_scenario])

    scenario(specs_path, calibrated_csv_path, results_folder, summary_folder, str(mled_scenario_number_counter) + "_" + mled_scenario)
    scenario_number_counter += 1

Running MLED scenario 'baseline'...
OnSSET Scenario: 1
Total electrification rate achieved in 2030 is 74.0 %
Total electrification rate achieved in 2060 is 99.8 %


2023-10-06 12:38:47,784		Finished


Running MLED scenario 'improved_access'...
OnSSET Scenario: 1
Total electrification rate achieved in 2030 is 87.0 %
Total electrification rate achieved in 2060 is 99.9 %


2023-10-06 12:41:47,710		Finished


Running MLED scenario 'ambitious_development'...
OnSSET Scenario: 1
Total electrification rate achieved in 2030 is 100.0 %
Total electrification rate achieved in 2060 is 100.0 %


2023-10-06 12:44:53,237		Finished


In [15]:
output_files_directory = results_folder
output_files_filenames = list(pathlib.Path(output_files_directory).glob('*.csv'))
scenario_output_files_dictionary = dict(zip(scenarios, output_files_filenames))

In [16]:
output_files_filenames

[WindowsPath('../results/onsset_full_results/1_baseline_zm-2-0_0_0_0_0_0.csv'),
 WindowsPath('../results/onsset_full_results/2_improved_access_zm-2-0_0_0_0_0_0.csv'),
 WindowsPath('../results/onsset_full_results/3_ambitious_development_zm-2-0_0_0_0_0_0.csv')]

In [17]:
scenario_output_files_dictionary

{'baseline': WindowsPath('../results/onsset_full_results/1_baseline_zm-2-0_0_0_0_0_0.csv'),
 'improved_access': WindowsPath('../results/onsset_full_results/2_improved_access_zm-2-0_0_0_0_0_0.csv'),
 'ambitious_development': WindowsPath('../results/onsset_full_results/3_ambitious_development_zm-2-0_0_0_0_0_0.csv')}

In [18]:
full_summary = pd.DataFrame()

for mled_scenario in scenarios:
    
    nest = pd.read_csv(os.path.join(scenario_output_files_dictionary[mled_scenario]))

    nest["BCU"] = nest["BCU"].astype("int")
    nest[["BCU","FinalElecCode2020","isurban_2020", "FinalElecCode2030","isurban_future_2030", "FinalElecCode2060","isurban_future_2060"]] = nest[["BCU","FinalElecCode2020","isurban_2020", "FinalElecCode2030","isurban_future_2030", "FinalElecCode2060","isurban_future_2060"]].astype("category")
    nest["isurban_future_2020"] = nest["isurban_2020"]

    nest.FinalElecCode2020 = nest.FinalElecCode2020.cat.set_categories([1,2,3,5,6,7,99])
    nest.FinalElecCode2030 = nest.FinalElecCode2030.cat.set_categories([1,2,3,5,6,7,99])
    nest.FinalElecCode2060 = nest.FinalElecCode2060.cat.set_categories([1,2,3,5,6,7,99])

    nest.FinalElecCode2020 = nest.FinalElecCode2020.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})
    nest.FinalElecCode2030 = nest.FinalElecCode2030.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})
    nest.FinalElecCode2060 = nest.FinalElecCode2060.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})

    nest.isurban_2020 = nest.isurban_2020.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2020 = nest.isurban_future_2020.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2030 = nest.isurban_future_2030.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2060 = nest.isurban_future_2060.cat.rename_categories({0: "rural", 1: "urban"})

    years = [2020, 2030, 2060]
    scenario_summary = pd.DataFrame()

    for year in years:
        yearly_summary = ((nest.groupby(["BCU","FinalElecCode"+str(year),"isurban_future_"+str(year)])["tot_dem_"+str(year)].sum()/10**6).round(2)).to_frame().reset_index()
        yearly_summary = yearly_summary.set_axis(["BCU", "tec", "urb_rur","value"], axis = 1)
        yearly_summary["year"] = year
        yearly_summary["scenario"] = mled_scenario
        yearly_summary["unit"] = "GWh"
        yearly_summary.insert(yearly_summary.shape[1]-1, "value", yearly_summary.pop("value"))

        scenario_summary = pd.concat([scenario_summary, yearly_summary])
        
    full_summary = pd.concat([full_summary, scenario_summary])
    
full_summary.to_csv(os.path.join(nest_summary_folder, "energy_allocation_results_for_nest.csv"), index = False)

In [19]:
full_summary

,BCU,tec,urb_rur,year,scenario,unit,value
0,1,grid_existing,rural,2020,baseline,GWh,5.2
1,1,grid_existing,urban,2020,baseline,GWh,0.0
2,1,grid_new,rural,2020,baseline,GWh,0.0
3,1,grid_new,urban,2020,baseline,GWh,0.0
4,1,sa_pv,rural,2020,baseline,GWh,0.0
...,...,...,...,...,...,...,...
331,24,mg_wind_hybrid,urban,2060,ambitious_development,GWh,0.0
332,24,mg_hydro,rural,2060,ambitious_development,GWh,0.0
333,24,mg_hydro,urban,2060,ambitious_development,GWh,0.0
334,24,unelectrified,rural,2060,ambitious_development,GWh,0.0


In [25]:
full_summary.groupby(["year","tec","scenario"]).value.sum().round(1).to_frame()

value
year tec            scenario                      
2020 grid_existing  ambitious_development   4246.0
                    baseline                4245.1
                    improved_access         4245.7
     grid_new       ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     sa_pv          ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     mg_pv_hybrid   ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     mg_wind_hybrid ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     mg_hydro       ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     unelectrified  ambitious_development   9252.5
                    baseline                9191.4
                    improved_access         9229.1
2030 grid_existing  ambitious_development  16891.0
                    baseline               16889.4
                    improved_access        16888.0
     grid_new       ambitious_development   8316.5
                    baseline                4942.0
                    improved_access         5145.5
     sa_pv          ambitious_development     35.0
                    baseline                  33.3
                    improved_access           35.9
     mg_pv_hybrid   ambitious_development   7944.8
                    baseline                1039.9
                    improved_access         3256.5
     mg_wind_hybrid ambitious_development      3.2
                    baseline                   2.6
                    improved_access            2.6
     mg_hydro       ambitious_development     38.9
                    baseline                  37.4
                    improved_access           36.0
     unelectrified  ambitious_development   4938.6
                    baseline               15130.6
                    improved_access        12520.8
2060 grid_existing  ambitious_development  31915.9
                    baseline               31909.3
                    improved_access        31912.9
     grid_new       ambitious_development  49445.8
                    baseline               32575.7
                    improved_access        33289.9
     sa_pv          ambitious_development     22.8
                    baseline                  65.1
                    improved_access           34.4
     mg_pv_hybrid   ambitious_development   1065.3
                    baseline                 522.3
                    improved_access          700.5
     mg_wind_hybrid ambitious_development      0.0
                    baseline                   0.0
                    improved_access            0.0
     mg_hydro       ambitious_development      5.1
                    baseline                   4.0
                    improved_access            4.4
     unelectrified  ambitious_development      0.0
                    baseline               17018.2
                    improved_access        16267.0